In [29]:
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG
from bokeh.plotting import figure, output_file, save
from matplotlib import category

from bybitapi import fetch_bybit_data_v5
from config import INTERVAL
import pandas as pd
import backtesting

In [30]:
category = 'spot'
pair = 'BTCUSDT'
end_date = datetime.datetime.now()
start_date = end_date -datetime.timedelta(1)

In [31]:


from TrainAndTest import loadmodel
from TrainingandValidation import TrainingAndValidation
from prep_data import prep_data

def IncIndicator(data,model) :
        """
        Returns `n`-period simple moving average of array `arr`.
        """
        return model.predict(data)

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        model = loadmodel("BTCUSDT","ensembleinc")
        data = self.data

        self.decision = self.I(IncIndicator,data,model)
        # close = self.data.Close
        # self.sma1 = self.I(SMA, close, self.n1)
        # self.sma2 = self.I(SMA, close, self.n2)
    
    def next(self):
        if self.decision==1:
            self.buy()
        else:
            self.sell()



class Train():
    trainingdata = []
        
    def get_data(self):
        
        #fetch the data
        data = fetch_bybit_data_v5(True,start_date,end_date,pair,INTERVAL,category)
        
        return data
    


In [32]:

train = Train()
data = train.get_data()
trainingdata = prep_data(data)

tv = TrainingAndValidation()

tv.train_and_cross_validate(trainingdata,"BTCUSDT",start_date,end_date,INTERVAL)

there are  1 total days in the range.
total_intervals: 288.0
intervals: [[1688625030.029167, 1688685030.029167], [1688685030.029167, 1688745030.029167]]
Fetching historical data from 2023-07-06 06:30:30.029167 to 2023-07-06 23:10:30.029167


Fetching historical data from 2023-07-06 23:10:30.029167 to 2023-07-07 15:50:30.029167


/workspace/bybitapi.py:99: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_new['timestamp'] = pd.to_datetime(df_new['open_time'], unit='ms')


returning new
                             open_time      open      high       low  \
timestamp                                                              
2023-07-06 06:35:48.608  1688625300000  29830.46  29863.48  29830.46   
2023-07-06 06:40:10.752  1688625600000  29863.47  29870.29  29863.47   
...                                ...       ...       ...       ...   
2023-07-07 06:24:29.696  1688711100000  30136.29  30154.06  30104.92   
2023-07-07 06:31:02.912  1688711400000   30143.7   30143.7  30140.25   

                            close    volume       turnover  
timestamp                                                   
2023-07-06 06:35:48.608  29863.47   0.13138  3922.36833465  
2023-07-06 06:40:10.752  29870.29  0.065886  1967.87736835  
...                           ...       ...            ...  
2023-07-07 06:24:29.696   30143.7  0.035206  1061.10714866  
2023-07-07 06:31:02.912  30140.25  0.013024   392.58472815  

[289 rows x 7 columns]
logging 2023-07-07 06:30:31.31

In [33]:

df = data.loc[:, ['open', 'high', 'low', 'close', 'volume']]
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
df.index = df.index.strftime('%Y-%m-%d %H:%M')
df.index = pd.to_datetime(df.index)
df = df.dropna()



In [36]:
print(df)
prepped_df = prep_data(df)
print(prepped_df)


                         Open      High       Low     Close    Volume
timestamp                                                            
2023-07-06 06:35:00  29830.46  29863.48  29830.46  29863.47   0.13138
2023-07-06 06:40:00  29863.47  29870.29  29863.47  29870.29  0.065886
...                       ...       ...       ...       ...       ...
2023-07-07 06:24:00  30136.29  30154.06  30104.92   30143.7  0.035206
2023-07-07 06:31:00   30143.7   30143.7  30140.25  30140.25  0.013024

[288 rows x 5 columns]


KeyError: 'close'

In [34]:

backtesting.set_bokeh_output(notebook=True)
bt = Backtest(df, SmaCross,
              cash=100000, commission=.002,
              exclusive_orders=True)
output = bt.run()
bt.plot()
print(output)

    


TypeError: '>' not supported between instances of 'str' and 'int'